<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/lab01_visualisation_de_la_tokenisation_et_des_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Lab 1: Visualisation de la tokenisation et des embeddings
---

**Objectif:**
- Comprendre comment un texte est segmenté en tokens et converti en vecteurs numériques(embeddings).
- Apprendre à visualiser et interpréter ces structures.

**Durée estimée:**
- 90–120 minutes

**Livrable :**
- Notebook avec graphiques présentant la segmentation en tokens et une visualisation 2D de vecteurs numériques.


---


## Step 1: Pré-requis (5 min)

Installation des librairies nécessaires :

In [ ]:
!pip install openai tiktoken matplotlib scikit-learn python-dotenv -q

### Configuration de la clé "OpenAI API Key" :

Puisque nous faisons ce notebook dans Google Colab, nos allons utiliser `userdata.get()` pour récupérer l'API key en toute sécurité :

Si cela n'a pas déjà été fait :
- Ouvre Google Colab → File → Upload notebook
- Ajoute ta clé API OpenAI :
  - Clique sur 🔑 (Secrets Manager)
  - Puis sur "+ Ajouter un secret"
  - Ajoute OPENAI_API_KEY et la valeur associée
  - Veiile à rendre le secret accessible depuis le notebook ( active "accès depuis le notebook")

In [ ]:
import os
from google.colab import userdata

# Get API key from Colab Secrets (add it in the Secrets manager: 🔑 icon on left panel)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Or if running locally with .env file:
# from dotenv import load_dotenv
# load_dotenv()

print("✓ API Key configured successfully")

---
## Step 2: Inspect Tokenization (15 min)

Use tiktoken (OpenAI's tokenizer) to break text into tokens:

In [ ]:
import tiktoken

# Initialize tokenizer for GPT-4o mini
enc = tiktoken.encoding_for_model("gpt-4o-mini")

# Sample text
sample_text = "Agentic AI agents can plan, reason, and use tools."

# Tokenize
tokens = enc.encode(sample_text)

# Display results
print("="*70)
print(f"Original text: {sample_text}")
print(f"Number of tokens: {len(tokens)}")
print(f"Token IDs: {tokens}")
print("\nDecoded tokens (individual):")
for i, token_id in enumerate(tokens, 1):
    decoded = enc.decode([token_id])
    print(f"  {i}. ID {token_id}: '{decoded}'")
print("="*70)

### Experiment with Different Text Types

Observe how punctuation, spaces, emojis, and special characters affect tokenization:

In [ ]:
test_texts = [
    "Hello, World!",
    "HelloWorld",
    "hello world",
    "AI is 🔥",
    "def my_function():\n    return True",
    "AGENTIC",
    "agentic",
]

print("\n" + "="*70)
print("TOKENIZATION ANALYSIS ACROSS DIFFERENT TEXT TYPES")
print("="*70)

for text in test_texts:
    tokens = enc.encode(text)
    decoded = [enc.decode([t]) for t in tokens]
    print(f"\nText: {repr(text)}")
    print(f"Tokens: {len(tokens)} → {tokens}")
    print(f"Decoded: {decoded}")

---
## Step 3: Compare Token Length Across Texts (10 min)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

sentences = [
    "AI is amazing.",
    "Artificial Intelligence is amazing.",
    "AI is 🔥.",
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog.",
]

# Collect data
data = []
for sentence in sentences:
    token_list = enc.encode(sentence)
    data.append({
        "Text": sentence,
        "Character Count": len(sentence),
        "Token Count": len(token_list),
        "Ratio (Chars/Token)": len(sentence) / len(token_list)
    })

df = pd.DataFrame(data)
print("\n" + "="*100)
print("TOKEN LENGTH COMPARISON")
print("="*100)
print(df.to_string(index=False))
print("="*100)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart: Character vs Token count
x = range(len(df))
axes[0].bar([i - 0.2 for i in x], df['Character Count'], width=0.4, label='Characters', color='steelblue')
axes[0].bar([i + 0.2 for i in x], df['Token Count'], width=0.4, label='Tokens', color='coral')
axes[0].set_xlabel('Sentence Index', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=11, fontweight='bold')
axes[0].set_title('Character Count vs Token Count', fontsize=12, fontweight='bold')
axes[0].set_xticks(x)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Line chart: Char/Token ratio
axes[1].plot(x, df['Ratio (Chars/Token)'], marker='o', linewidth=2, markersize=8, color='green')
axes[1].set_xlabel('Sentence Index', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Ratio (Chars/Token)', fontsize=11, fontweight='bold')
axes[1].set_title('Character-to-Token Ratio', fontsize=12, fontweight='bold')
axes[1].set_xticks(x)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Insight: Emojis and special symbols tend to use more tokens per character!")

---
## Step 4: Generate Embeddings (20 min)

Use OpenAI's `text-embedding-3-small` model to create numerical vectors:

In [ ]:
from openai import OpenAI
import numpy as np

# Initialize OpenAI client
client = OpenAI()

# Texts to embed
texts = [
    "Agentic AI",
    "Autonomous agents",
    "Bananas are yellow",
    "Machine learning models",
    "Fruit is delicious",
]

embeddings = []

print("\n" + "="*70)
print("GENERATING EMBEDDINGS")
print("="*70)

for text in texts:
    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    vector = resp.data[0].embedding
    embeddings.append(vector)
    print(f"✓ '{text}' → Vector length: {len(vector)}")

print("="*70)
print(f"\n📊 Embedding Dimension: {len(embeddings[0])}")
print(f"Number of texts embedded: {len(embeddings)}")

# Show a sample embedding (first 20 values)
print(f"\nSample embedding (first 20 values) for '{texts[0]}':")
print(embeddings[0][:20])

---
## Step 5: Dimensionality Reduction & Plotting (25 min)

Reduce 1536-dimensional embeddings to 2D for visualization:

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Apply PCA to reduce to 2D
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Calculate explained variance
explained_var = pca.explained_variance_ratio_
total_var = sum(explained_var)

print(f"\n📈 PCA Explained Variance:")
print(f"  PC1: {explained_var[0]:.4f} ({explained_var[0]*100:.2f}%)")
print(f"  PC2: {explained_var[1]:.4f} ({explained_var[1]*100:.2f}%)")
print(f"  Total: {total_var:.4f} ({total_var*100:.2f}%)")

# Create visualization
plt.figure(figsize=(10, 8))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

for i, (txt, color) in enumerate(zip(texts, colors)):
    x, y = embeddings_2d[i]
    plt.scatter(x, y, s=300, alpha=0.7, color=color, edgecolors='black', linewidth=2)
    plt.text(x + 0.05, y + 0.05, txt, fontsize=10, fontweight='bold',
             bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.3))

plt.xlabel(f'PC1 ({explained_var[0]*100:.1f}%)', fontsize=12, fontweight='bold')
plt.ylabel(f'PC2 ({explained_var[1]*100:.1f}%)', fontsize=12, fontweight='bold')
plt.title('Embedding Visualization (PCA 2D)', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.tight_layout()
plt.show()

print("\n💡 Observation: Semantically similar texts should be closer together!")

### Interactive Embedding Explorer

Add more sentences and visualize clusters:

In [ ]:
# Expand with more related texts
extended_texts = texts + [
    "AI agents with reasoning",
    "Yellow banana fruit",
    "Deep learning neural networks",
    "Apples are red",
]

# Generate embeddings for new texts
extended_embeddings = []

print("\nGenerating embeddings for extended set...")
for text in extended_texts:
    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    extended_embeddings.append(resp.data[0].embedding)

# Apply PCA
pca_ext = PCA(n_components=2)
embeddings_2d_ext = pca_ext.fit_transform(extended_embeddings)

# Categorize texts
categories = {
    'AI/Agents': [0, 1, 3, 5, 6],
    'Food/Fruit': [2, 4, 7, 8],
}

category_colors = {'AI/Agents': '#FF6B6B', 'Food/Fruit': '#4ECDC4'}

# Visualize
fig, ax = plt.subplots(figsize=(12, 9))

for category, indices in categories.items():
    for idx in indices:
        x, y = embeddings_2d_ext[idx]
        ax.scatter(x, y, s=400, alpha=0.7, color=category_colors[category],
                  edgecolors='black', linewidth=2, label=category if idx == indices[0] else '')
        ax.text(x + 0.08, y + 0.08, extended_texts[idx], fontsize=9, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.3', facecolor=category_colors[category], alpha=0.3))

ax.set_xlabel(f'PC1 ({pca_ext.explained_variance_ratio_[0]*100:.1f}%)', fontsize=12, fontweight='bold')
ax.set_ylabel(f'PC2 ({pca_ext.explained_variance_ratio_[1]*100:.1f}%)', fontsize=12, fontweight='bold')
ax.set_title('Extended Embedding Visualization - Semantic Clusters', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)

# Remove duplicate labels
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), fontsize=11, loc='best')

plt.tight_layout()
plt.show()

---
## Step 6: Experiment with Semantic Similarity (15 min)

Compute cosine similarity between embeddings:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Use original 5 texts for clarity
similarity_matrix = cosine_similarity(embeddings)

# Create DataFrame for better visualization
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=texts,
    columns=texts
)

print("\n" + "="*100)
print("COSINE SIMILARITY MATRIX")
print("="*100)
print(similarity_df.round(4))
print("="*100)

# Visualization
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(similarity_matrix, cmap='coolwarm', vmin=0, vmax=1)

ax.set_xticks(range(len(texts)))
ax.set_yticks(range(len(texts)))
ax.set_xticklabels(texts, rotation=45, ha='right')
ax.set_yticklabels(texts)

# Add values to heatmap
for i in range(len(texts)):
    for j in range(len(texts)):
        text = ax.text(j, i, f'{similarity_matrix[i, j]:.3f}',
                       ha="center", va="center", color="black", fontweight='bold')

ax.set_title('Cosine Similarity Between Embeddings', fontsize=14, fontweight='bold', pad=20)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Similarity Score', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# Find most and least similar pairs
print("\n🔍 Key Similarities:")
similarities_list = []
for i in range(len(texts)):
    for j in range(i+1, len(texts)):
        similarities_list.append((texts[i], texts[j], similarity_matrix[i, j]))

similarities_list.sort(key=lambda x: x[2], reverse=True)

print("\n  🏆 Most Similar:")
for text1, text2, score in similarities_list[:3]:
    print(f"    '{text1}' ↔ '{text2}': {score:.4f}")

print("\n  ❄️  Least Similar:")
for text1, text2, score in similarities_list[-3:]:
    print(f"    '{text1}' ↔ '{text2}': {score:.4f}")

---
## Step 7: Mini-Project - Token & Embedding Explorer (15–20 min)

Build a reusable function to analyze any text:

In [ ]:
class TextAnalyzer:
    """Comprehensive text tokenization and embedding analyzer."""

    def __init__(self, reference_text=None):
        self.enc = tiktoken.encoding_for_model("gpt-4o-mini")
        self.client = OpenAI()
        self.reference_text = reference_text
        self.reference_embedding = None

        if reference_text:
            resp = self.client.embeddings.create(
                model="text-embedding-3-small",
                input=reference_text
            )
            self.reference_embedding = resp.data[0].embedding

    def tokenize(self, text):
        """Tokenize text and return tokens with decoded values."""
        token_ids = self.enc.encode(text)
        token_strs = [self.enc.decode([t]) for t in token_ids]
        return token_ids, token_strs

    def embed(self, text):
        """Generate embedding for text."""
        resp = self.client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        return resp.data[0].embedding

    def analyze(self, text):
        """Comprehensive analysis of input text."""
        token_ids, token_strs = self.tokenize(text)
        embedding = self.embed(text)

        result = {
            'text': text,
            'char_count': len(text),
            'token_count': len(token_ids),
            'tokens': list(zip(token_ids, token_strs)),
            'embedding': embedding,
            'embedding_dim': len(embedding)
        }

        # Compute similarity to reference if available
        if self.reference_embedding:
            similarity = cosine_similarity(
                [embedding],
                [self.reference_embedding]
            )[0][0]
            result['similarity_to_reference'] = similarity

        return result

    def display_analysis(self, text):
        """Print formatted analysis."""
        analysis = self.analyze(text)

        print("\n" + "="*80)
        print(f"TEXT: '{analysis['text']}'")
        print("="*80)
        print(f"Character Count: {analysis['char_count']}")
        print(f"Token Count: {analysis['token_count']}")
        print(f"Embedding Dimension: {analysis['embedding_dim']}")

        if 'similarity_to_reference' in analysis:
            print(f"\nSimilarity to Reference ('{self.reference_text}'): {analysis['similarity_to_reference']:.4f}")

        print("\nTokens:")
        for i, (token_id, token_str) in enumerate(analysis['tokens'], 1):
            print(f"  {i:2d}. ID {token_id:5d} → '{token_str}'")

        print(f"\nEmbedding Preview (first 10 values): {analysis['embedding'][:10]}")
        print("="*80)

# Initialize analyzer with a reference text
analyzer = TextAnalyzer(reference_text="AI and agents")

# Test with different texts
test_sentences = [
    "Agentic AI systems are intelligent.",
    "Machine learning requires data.",
    "I love bananas 🍌",
]

for sentence in test_sentences:
    analyzer.display_analysis(sentence)

### Test with Your Own Texts

Try adding your custom sentences below:

In [ ]:
# Add your own sentences here for analysis!
custom_texts = [
    "Your sentence here",
    "Another sentence",
    "Add as many as you want",
]

for text in custom_texts:
    analyzer.display_analysis(text)

---
## Step 8: Advanced Visualization - Similarity Network

Create a network graph showing similarity relationships:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyArrowPatch
import numpy as np

# Use extended_texts from earlier
similarity_matrix_ext = cosine_similarity(extended_embeddings)

# Create network visualization
fig, ax = plt.subplots(figsize=(14, 10))

# Use 2D embedding positions as node positions
positions = embeddings_2d_ext

# Draw edges (similarity links) - only if similarity > threshold
threshold = 0.7
for i in range(len(extended_texts)):
    for j in range(i+1, len(extended_texts)):
        sim = similarity_matrix_ext[i, j]
        if sim > threshold:
            x1, y1 = positions[i]
            x2, y2 = positions[j]
            # Line opacity and width based on similarity
            alpha = (sim - threshold) / (1 - threshold) * 0.8
            width = 1 + (sim - threshold) / (1 - threshold) * 3
            ax.plot([x1, x2], [y1, y2], 'k-', alpha=alpha, linewidth=width)

# Draw nodes
for idx, (text, (x, y)) in enumerate(zip(extended_texts, positions)):
    category = 'AI/Agents' if idx in categories['AI/Agents'] else 'Food/Fruit'
    color = category_colors[category]
    ax.scatter(x, y, s=500, alpha=0.8, color=color, edgecolors='black', linewidth=2, zorder=3)
    ax.text(x, y, str(idx), ha='center', va='center', fontsize=9, fontweight='bold', zorder=4)

# Add legend with text labels
ax.text(0.02, 0.98, 'Node Labels:', transform=ax.transAxes, fontsize=11, fontweight='bold',
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

label_text = '\n'.join([f"{i}: {text}" for i, text in enumerate(extended_texts)])
ax.text(0.02, 0.92, label_text, transform=ax.transAxes, fontsize=8, verticalalignment='top',
        family='monospace', bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.3))

# Add category legend
ai_patch = mpatches.Patch(color='#FF6B6B', label='AI/Agents', edgecolor='black')
food_patch = mpatches.Patch(color='#4ECDC4', label='Food/Fruit', edgecolor='black')
ax.legend(handles=[ai_patch, food_patch], loc='upper right', fontsize=11, framealpha=0.9)

ax.set_xlabel(f'PC1 ({pca_ext.explained_variance_ratio_[0]*100:.1f}%)', fontsize=12, fontweight='bold')
ax.set_ylabel(f'PC2 ({pca_ext.explained_variance_ratio_[1]*100:.1f}%)', fontsize=12, fontweight='bold')
ax.set_title(f'Semantic Similarity Network (threshold > {threshold})', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

print(f"\n📊 Network Statistics (Threshold: {threshold}):")
connection_count = 0
for i in range(len(extended_texts)):
    for j in range(i+1, len(extended_texts)):
        if similarity_matrix_ext[i, j] > threshold:
            connection_count += 1
            print(f"  [{i}] '{extended_texts[i]}' ↔ [{j}] '{extended_texts[j]}': {similarity_matrix_ext[i, j]:.4f}")

print(f"\nTotal connections: {connection_count}")

---
## Summary & Key Takeaways

### What We Learned:

1. **Tokenization** breaks text into discrete units that models understand
   - Different texts have different token-to-character ratios
   - Emojis and special characters use more tokens

2. **Embeddings** convert tokens into high-dimensional numerical vectors
   - OpenAI's `text-embedding-3-small` produces 1536-dimensional vectors
   - These vectors capture semantic meaning

3. **Dimensionality Reduction** (PCA) allows us to visualize embeddings
   - We can reduce 1536D → 2D while preserving semantic relationships
   - Semantically similar texts cluster together

4. **Cosine Similarity** measures semantic closeness
   - Scores range from 0 (opposite) to 1 (identical)
   - Used in RAG, semantic search, and LLM applications

### Applications:

- **RAG (Retrieval-Augmented Generation)**: Find relevant documents by embedding similarity
- **Semantic Search**: Find similar texts without keyword matching
- **Clustering**: Group semantically similar documents
- **Recommendation Systems**: Suggest similar items based on embeddings
- **LLM Reasoning**: Models use embeddings internally for reasoning

### Next Steps:

- Build a simple RAG pipeline with vector databases (Pinecone, Weaviate)
- Experiment with different embedding models
- Create semantic search applications
- Explore advanced tokenization for multilingual texts

---

**🎓 Lab Complete! You now understand how text becomes numbers in LLMs.**